In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 344.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.8/871.8 kB 10.8 MB/s eta 0:00:00


In [2]:
from ultralytics import YOLO

MODEL_PATH = '/content/yolov8n.pt'

model = YOLO(MODEL_PATH)

In [3]:
model

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

In [4]:
def get_person(class_names_dict, target_class='person'):
    for key, val in class_names_dict.items():
        if val == target_class:
            return key

In [5]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

VIDEO_LOC = '/content/Demo Video 1.mp4'

video_capture = cv2.VideoCapture(VIDEO_LOC)

while video_capture.isOpened():
    success, frame = video_capture.read()
    if success:

        result_set = model(frame)[0]

        person_id = get_person(result_set.names)
        person_img_masked = [res for res in result_set if res.boxes.cls == person_id]

        num_people = len(person_img_masked)

        for person in person_img_masked:

            box = person.boxes.xyxy[0].numpy()
            x_min, y_min, x_max, y_max = map(int, box)

            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

            cv2.putText(frame, 'Person', (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

        cv2.putText(frame, f'People in frame: {num_people}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

        cv2_imshow(frame)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

video_capture.release()

cv2.destroyAllWindows()

In [ ]:
# # Function to get the person_id from the classes identifiable by the model
# def get_person(class_names_dict, target_class='person'):
#     for key, val in class_names_dict.items():
#         if val == target_class:
#             return key
# # 8.
# def get_overlap_mask(annotated_frame, person_region, x1, y1, x2, y2):
#     masked_img_map = np.zeros(annotated_frame.shape) # Zeros for all image
#     masked_img_map[int(person_region[1]):int(person_region[3]), int(person_region[0]):int(person_region[2])] = 1 # fill 1's where the person is
#     overlap_reg_map = np.zeros(annotated_frame.shape) # Zeros for all image
#     overlap_reg_map[y1:y2, x1:x2] = 1 # fill 1's where the restricted area is
#     return overlap_reg_map

# # 9.
# def get_iou(masked_img_map, overlap_reg_map):
#     union = masked_img_map + overlap_reg_map # Calculate the union of the person and the restricted area
#     overlap = masked_img_map * overlap_reg_map # Calculate the intersection of the person and the restricted area
#     IOU = overlap.sum()/float(union.sum()) # Get the IOU value

# import cv2
# import numpy as np

# VIDEO_LOC = '/kaggle/input/yolo-demo/TEST_VIDEO.mp4'

# # 1.
# enter_x1, enter_y1, enter_x2, enter_y2 = (750, 0, 800, 478)
# exit_x1, exit_y1, exit_x2, exit_y2 = (200, 0, 250, 478)

# # 2.
# video_capture = cv2.VideoCapture(VIDEO_LOC)
# # video_capture = cv2.VideoCapture(0) # For webcam test

# enter_count, exit_count = 0, 0
# entering, exiting = False, False

# while video_capture.isOpened():
#     success, frame = video_capture.read()
#     if success: # Check if the frame is read successfully
#         # 3.
#         result_set = model(frame)[0]

#         # 4. Retain the result of person ONLY.
#         person_id = get_person(result_set.names)
#         # 5.
#         person_img_masked = [res for res in result_set if res.boxes.cls == person_id]
#         if len(person_img_masked) < 1:
#             # 5.A.
#             continue
#         else:
#             # 5.B.
#             person_img_masked = person_img_masked[0]
#         # 5.B.
#         person_region = person_img_masked.boxes.xyxy[0].numpy()

#         # 6.
#         annotated_frame = person_img_masked.plot()

#         # 7.
#         region_img = annotated_frame[enter_y1:enter_y2, enter_x1:enter_x2]
#         region = np.zeros(region_img.shape, dtype=np.uint8)
#         res = cv2.addWeighted(region, 0.7, region_img, 0.5, 1.0)
#         annotated_frame[y1:y2, x1:x2] = res
#         region_img = annotated_frame[exit_y1:exit_y2, exit_x1:exit_x2]
#         region = np.zeros(region_img.shape, dtype=np.uint8)
#         res = cv2.addWeighted(region, 0.7, region_img, 0.5, 1.0)
#         annotated_frame[y1:y2, x1:x2] = res

#         # 8.
#         if entering:
#             overlap_reg_map = get_overlap_mask(annotated_frame, person_region, exit_x1, exit_y1, exit_x2, exit_y2)
#             exit_iou = get_iou(masked_img_map, overlap_reg_map)
#             if exit_iou > 0.5:
#                 entering = False
#                 enter_count += 1
#         elif exitting:
#             overlap_reg_map = get_overlap_mask(annotated_frame, person_region, enter_x1, enter_y1, enter_x2, enter_y2)
#             enter_iou = get_iou(masked_img_map, overlap_reg_map)
#             if enter_iou > 0.5:
#                 exitting = False
#                 exit_count += 1
#         else:
#             overlap_enter_map = get_overlap_mask(annotated_frame, person_region, enter_x1, enter_y1, enter_x2, enter_y2)
#             overlap_exit_map = get_overlap_mask(annotated_frame, person_region, exit_x1, exit_y1, exit_x2, exit_y2)

#             enter_iou = get_iou(masked_img_map, overlap_enter_map)
#             exit_iou = get_iou(masked_img_map, overlap_exit_map)

#             if enter_iou > exit_iou:
#                 entering = True
#             elif enter_iou < exit_iou:
#                 exitting = True

#         # 10.
#         cv2.putText(annotated_frame, f'COUNTER entered: {enter_count}, exitted: {exit_count}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

#         frame = annotated_frame

#         # Display the resulting frame
#     cv2.imshow('PROJECT', frame)

#     # Press Q on keyboard to exit
#     if cv2.waitKey(25) & 0xFF == ord('q'):
#         break

# # When everything done, release
# # the video capture object
# video_capture.release()

# # Closes all the frames
# cv2.destroyAllWindows()